## <ins>Filtrage collaboratif : approche mémoire</ins>

Le filtrage collaboratif est rapidement devenu une des techniques les plus utilisées dans la construction des systèmes de recommandation.

Le site Amazon, dont l'activité initiale ne concernait que la vente de livres en ligne, a popularisé cette méthode avec son service "Les personnes qui ont aimé ce livre ont également aimé ce livre".

Le filtrage collaboratif consiste à fournir des recommandations en exploitant exclusivement les interactions passées dans les utilisateurs et les produits, en regroupant et identifiant des groupes d'utilisateurs ou de produits dont les interactions sont similaires.

Plus précisément, le filtrage collaboratif utilise une matrice de notations dont les lignes correspondent aux utilisateurs (users) et les colonnes aux éléments (items). Ainsi, la cellule (u,i) de la matrice correspond soit au score donné par l'utilisateur u au produit i soit à une case vide si l'utilisateur n'a pas eu d'interaction avec le produit.

Ce score associé à un utilisateur et à un produit peut être recueilli de deux manières :

Explicite : l'utilisateur attribue des notes aux produits testés ou signale explicitement ses préférences (likes/dislikes)
Implicite : le score est calculé en fonction des interactions de l'utilisateur avec ce produit (vu, mis dans le panier, acheté, durée sur la page...)
Le but du filtrage collaboratif est de prédire le score qu'attribuerait un utilisateur à un produit avec lequel il n'a pas encore testé. Ainsi, plus la prédiction est bonne plus le produit sera pertinent à recommander à l'utilisateur.

La technique du filtrage collaboratif peut être mise oeuvre selon deux approches que nous allons explorer : l’approche mémoire et l’approche modèle.

Dans ce notebook, nous aborderons seulement l'approche mémoire . L'approche modèle sera vu en détail dans le 3ème notebook.

In [2]:
import pandas as pd

In [9]:
ratings = pd.read_csv('/home/antoine/PROJET_MLOPS_RECO_MOVIES/data/raw/silver/processed_ratings.csv')
ratings.head()
print(ratings.shape)

,userid,movieid,rating,timestamp,bayesian_mean
0,1,2,3.5,1112486027,3.21
1,1,29,3.5,1112484676,3.89
2,1,32,3.5,1112484819,3.89
3,1,47,3.5,1112484727,4.04
4,1,50,3.5,1112484580,4.32


In [10]:
movies = pd.read_csv('/home/antoine/PROJET_MLOPS_RECO_MOVIES/data/raw/silver/processed_movies.csv')

movies.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 5 columns):
 #   Column         Dtype  
---  ------         -----  
 0   userid         object 
 1   movieid        object 
 2   rating         float64
 3   timestamp      int64  
 4   bayesian_mean  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 762.9+ MB


In [11]:
df = pd.merge(ratings, movies, on='movieid')
df.head()

,movieid,title,genres,year
0,1,Toy Story,"Adventure, Animation, Children, Comedy, Fantasy",1995
1,2,Jumanji,"Adventure, Children, Fantasy",1995
2,3,Grumpier Old Men,"Comedy, Romance",1995
3,4,Waiting to Exhale,"Comedy, Drama, Romance",1995
4,5,Father of the Bride Part II,Comedy,1995


Une fois la jointure fait les variables qui vont nous intéresser sont : userid, title et rating . Finalement, on fera un filtrage pour garder seulement les utilisateurs ayant noté au moins 10 films et les films étant notés au moins 2 fois. Cela nous permet d'avoir un dataset plus facile à manipuler.

In [13]:
# Filtrer les utilisateurs ayant noté au moins 20 films
user_counts = df['userid'].value_counts()
users_to_keep = user_counts[user_counts >= 20].index
df_filtered_users = df[df['userid'].isin(users_to_keep)]

# Filtrer les films ayant été notés au moins 4 fois
movie_counts = df_filtered_users['movieid'].value_counts()
movies_to_keep = movie_counts[movie_counts >= 4].index
df = df_filtered_users[df_filtered_users['movieid'].isin(movies_to_keep)]

# Afficher le DataFrame filtré
df.head()

,userid,movieid,rating,timestamp,bayesian_mean,title,genres,year
0,1,2,3.5,1112486027,3.21,Jumanji,"Adventure, Children, Fantasy",1995
1,5,2,3.0,851527569,3.21,Jumanji,"Adventure, Children, Fantasy",1995
2,13,2,3.0,849082742,3.21,Jumanji,"Adventure, Children, Fantasy",1995
3,29,2,3.0,835562174,3.21,Jumanji,"Adventure, Children, Fantasy",1995
4,34,2,3.0,846509384,3.21,Jumanji,"Adventure, Children, Fantasy",1995


In [6]:
df.shape

(19988140, 8)

### <ins>User-based filtering</ins>


L'approche mémoire se base sur la corrélation entre les comportements "passés" des utilisateurs

Pour cela, elle se base sur sur une matrice de notations où les lignes représentent les utilisateurs et les colonnes représentent le contenu, ici les films.

Ainsi, la cellule (u,i) de la matrice de notations correspond alors à la note donnée par l'utilisateur u au contenu i.

Ces interactions peuvent être analysées de deux manières :

* Soit l'analyse est basée sur les utilisateurs, (user-based filtering)
* Soit l'analyse est basée sur les produits, (item-based filtering)

Illustrons ces deux approches :



Cette approche repose sur l'idée qu'un utilisateur est susceptible d'avoir les mêmes préférences qu'un utilisateur ayant le même comportement de notation que lui

### <ins>Item-based filtering</ins>

Cette approche se base cette fois sur l'idée que deux films qui ont reçu les mêmes notes par un même utilisateur sont susceptible d'être similaires.

#### Création de la matrice de notations

Nous allons construire la matrice de notations où chaque ligne représente les notes données par un utilisateur et chaque colonne les notes attribuées à un contenu.

Ainsi, la cellule (u,i) de la matrice modèle correspond alors à la note donnée par l'utilisateur u au contenu i.

In [21]:
n_users = df['userid'].nunique()

n_books = df['title'].nunique()
print("Nombre d'utilisateurs : ", n_users, end="\n\n")
print("Nombre de livres : ", n_books)

In [22]:
n_users = df['userid'].nunique()

n_books = df['title'].nunique()
print("Nombre d'utilisateurs : ", n_users, end="\n\n")
print("Nombre de livres : ", n_books)

In [23]:
df.info()

: 

In [18]:
df = df.astype({'userid': 'int32', 'rating': 'int32', 'movieid': 'int32'})

df = df.sample(frac=0.15, random_state=42).reset_index(drop=True)

title,"""Great Performances"" Cats",$ (Dollars),$9.99,'71,'Round Midnight,'Salem's Lot,'Til There Was You,"'burbs, The",'night Mother,(500) Days of Summer,...,Zozo,Zulu,[REC],[REC]²,eXistenZ,loudQUIETloud: A Film About the Pixies,xXx,xXx: State of the Union,¡Three Amigos!,À nous la liberté (Freedom for Us)
userid,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
mat_ratings = mat_ratings +1

Dans les prochaines étapes, notre objectif est de substituer les valeurs manquantes dans cette matrice par des zéros. Cependant, il est essentiel de ne pas fausser notre système de notation, qui commence également à partir de 0.
Nous augmentons ainsi nos notes de 1

In [ ]:
mat_ratings = mat_ratings +1